# Component: Model I/O

In [141]:
# !pip install langchain langchain-openai

In [142]:
from dotenv import load_dotenv     # dotenv 파일에서 환경변수 로드
load_dotenv()   # .env 파일에서 환경변수 로드

True

In [ ]:
import os   # 환경변수 사용을 위한 모듈
os.getenv("HF_TOKEN")

### PromptTemplate

In [144]:
# ProptTemplate
from langchain import PromptTemplate    # 프롬프트 템플릿을 사용하기 위한 모듈

template = "{product}를 홍보하기 위한 재미있고, 신선한 광고문구를 작성해 주세요"    # 프롬프트 템플릿 정의

# PromptTemplate 객체 생성
prompt = PromptTemplate(
    template=template,  # 프롬프트 템플릿
    input_variables=['product'] # 입력 변수 정의
)

print(prompt.format(product='자동차'))  # 프롬프트 템플릿에 '자동차'를 넣어 출력

자동차를 홍보하기 위한 재미있고, 신선한 광고문구를 작성해 주세요


In [145]:
# FewShotPromptTemplate
from langchain.prompts import FewShotPromptTemplate # FewShotPromptTemplate을 사용하기 위한 라이브러리

# FewShotPromptTemplate은 예시를 제공하여 모델이 더 나은 결과를 생성하도록 돕는 템플릿
examples = [
    {"question": "2 + 2는 무엇인가요?", "answer": "2 + 2 = 4"}, 
    {"question": "3 + 4는 무엇인가여?", "answer": "3 + 4 = 7"}
]

# 예시를 프롬프트 템플릿에 맞게 변환
example_prompt = PromptTemplate(
    template="Q: {question}\nA: {answer}",  # 예시 프롬프트 템플릿
    input_varialbes=['question', 'answer']  # 입력 변수 정의
)

# 예시 프롬프트 템플릿을 사용하여 예시를 생성
fewshot_prompt = FewShotPromptTemplate(
    examples=examples,  # 예시 리스트
    example_prompt=example_prompt,  # 예시 프롬프트 템플릿
    prefix='다음 계산 문제를 해결하세요',   # 프롬프트 접두사
    suffix="Q: {question}은 무엇인가요?\nA:",   # 프롬프트 접미사
    input_variables=['question']    # 입력 변수 정의
)

print(fewshot_prompt.format(question='24 + 78'))

다음 계산 문제를 해결하세요

Q: 2 + 2는 무엇인가요?
A: 2 + 2 = 4

Q: 3 + 4는 무엇인가여?
A: 3 + 4 = 7

Q: 24 + 78은 무엇인가요?
A:


In [146]:
# ChatPromptTemplate = System, Human, AI 유형별 메세지 작성

from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

sys_msg = SystemMessagePromptTemplate.from_template('당신은 친절한 쳇봇입니다.')    # 시스템 메시지 템플릿 생성
hm_msg = HumanMessagePromptTemplate.from_template('질문: {question}')   # 사용자 메시지 템플릿 생성
msg = ChatPromptTemplate.from_messages([sys_msg, hm_msg])   # 챗 프롬프트 템플릿 생성

msg.format_messages(question='AI를 배우려면 무엇부터 해야 하나요?') # 메시지 포맷팅

[SystemMessage(content='당신은 친절한 쳇봇입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='질문: AI를 배우려면 무엇부터 해야 하나요?', additional_kwargs={}, response_metadata={})]

### OutputParser

In [147]:
# Outputparser 인스턴스 및 promptTemplate 인스턴스 생성
from langchain_openai import ChatOpenAI   # OpenAI 챗 모델을 사용하기 위한 라이브러리
from langchain.output_parsers import CommaSeparatedListOutputParser   # 출력 파서 라이브러리

output_parser = CommaSeparatedListOutputParser()   # 쉼표로 구분된 리스트 출력 파서 생성
format_instructions = output_parser.get_format_instructions()   # 출력 포맷 지침 확인

prompt_tpl = PromptTemplate(
    template="{subject} 5개의 팀을 알려주세요.\n형식 지정:{format}",    # 프롬프트 템플릿
    input_variables=['subject'],                                      # 사용자 입력 변수 정의
    partial_variables={'format': format_instructions},                # 고정 설정 변수 정의
)

In [148]:
# 프롬프트 생성
query = "한국의 프로야구팀" # 사용자 입력값

prompt = prompt_tpl.format(subject=query)   # 프롬프트 템플릿에 사용자 입력값을 넣어 포맷팅
print(prompt)   # 포맷팅된 프롬프트 출력

한국의 프로야구팀 5개의 팀을 알려주세요.
형식 지정:Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [149]:
# openAI api 모델 사용을 위한 인스턴스 생성 및 요청 처리
model = ChatOpenAI(
    model_name="gpt-4o-mini",   # 사용할 모델 이름
    temperature=0,   # 생성된 텍스트의 다양성 조절 (0: 결정적, 1: 무작위적)
    max_tokens=2048,   # 생성할 최대 토큰 수
)

model.invoke(prompt)   # 모델에 프롬프트(질의)를 전달하여 응답 생성

response = model.invoke(prompt)   # 모델에 프롬프트(질의)를 전달하여 응답 생성

In [150]:
# response   # 모델의 응답 출력
print(response.content)   # 모델의 응답 내용 출력
print(type(response.content))   # 응답 객체의 타입 확인 string

두산 베어스, LG 트윈스, 삼성 라이온즈, 키움 히어로즈, NC 다이노스
<class 'str'>


In [151]:
print(output_parser.parse(response.content))   # 응답 내용을 파싱하여 리스트 형태로 출력
print(type(output_parser.parse(response.content)))   # 파싱된 응답의 타입 확인 string -> list

['두산 베어스', 'LG 트윈스', '삼성 라이온즈', '키움 히어로즈', 'NC 다이노스']
<class 'list'>


### HuggingFace 모델 사용하기

In [152]:
# !pip install langchain langchain_huggingface

In [153]:
from langchain_huggingface import HuggingFaceEndpoint   # HuggingFace 모델을 사용하기 위한 라이브러리
from langchain_huggingface import ChatHuggingFace   # HuggingFace 챗 모델을 사용하기 위한 라이브러리

# HuggingFaceEndpoint 인스턴스 생성
# HuggingFaceEndpoint는 Hugging Face의 모델을 API로 호출하기 위한 클래스
end_point = HuggingFaceEndpoint(
    repo_id="Bllossom/llama-3.2-Korean-Bllossom-3B",    # 사용할 모델의 레포지토리 ID
    task="text-generation", # 모델의 작업 유형 (text-generation: 텍스트 생성)
    max_new_tokens=1024,    # 생성할 최대 토큰 수
)

# ChatHuggingFace 인스턴스 생성
# ChatHuggingFace는 Hugging Face의 챗 모델을 사용하기 위한 클래스
hf_model = ChatHuggingFace(
    llm=end_point,  # HuggingFaceEndpoint 인스턴스를 LLM으로 사용
    verbose=True    # 모델의 응답을 자세히 출력할지 여부 (True: 자세히, False: 간단히)
)

hf_model.invoke(prompt)   # 모델에 프롬프트(질의)를 전달하여 응답 생성
response = hf_model.invoke('저는 아침으로 사과를 먹었습니다. 저는 아침에 뭘 먹었을까요?')   # 모델에 프롬프트(질의)를 전달하여 응답 생성

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


HfHubHTTPError: 503 Server Error: Service Temporarily Unavailable for url: https://router.huggingface.co/hf-inference/models/Bllossom/llama-3.2-Korean-Bllossom-3B/v1/chat/completions

In [ ]:
from langchain.modlel_laberathy import ModelLaboratory     # 모델 실험을 위한 라이브러리

model_lab = ModelLaboratory.from_llms([model, hf_model])   # 모델 실험실 인스턴스 생성
model_lab.compare('아침에 사과를 먹는 것의 효과를 알려주세요.')   # 모델 실험실 인스턴스 생성

ModuleNotFoundError: No module named 'langchain.modlel_laberathy'